In [1]:
###Importing the important libraries needed for Data Manipulation, classification and Analysis

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 
from sklearn.datasets import load_digits 
digits = load_digits() 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics

from sklearn.impute import SimpleImputer

In [18]:
# Import data using predefined file path
data_file1 = "train.csv" 
df_train= pd.read_csv(data_file1)
# Importing data using user defined file path
df_test = pd.read_csv('test.csv')
df_gender_submission = pd.read_csv('gender_submission.csv')

# Merge the dataset using PassengerId as a key and using right join 
# (all the rows from df_gender_submission and only matching rows from df_test)
result = pd.merge(df_gender_submission, df_test, how = "right", on = ["PassengerId"])
# Concatenating df_train and result dataframe
train_df = pd.concat([df_train, result], ignore_index=True)

# Print first few rows
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [19]:
train_df.head(10) 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [20]:
# Check data types
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [16]:
train_df.shape

(891, 12)

In [21]:
# check the unique values count
print("Number of unique values in each columns")
for col in train_df.columns:
    print(str(col)+ "   " + str(train_df[col].nunique()))

print('''It is clear when look at the number of unique values that there are\n
•\t {train_df.PassengerId.nunique()} unique values in the PassengerId column. 
However, only {train_df.PassengerId.nunique()} unique values in the Name column. 
This raise the suspision that there may be duplicate entry in the data set''')

Number of unique values in each columns
PassengerId   1309
Survived   2
Pclass   3
Name   1307
Sex   2
Age   98
SibSp   7
Parch   8
Ticket   929
Fare   281
Cabin   186
Embarked   3


In [ ]:
#TODO %%%%%%%%%%%%%% Need to write code to check and remove duplicate values

In [25]:
print("\tData description") 
print("*****************************************") 

''' Within the three quatition (i.e """ """), it is posible to display a pragraph as it is.
Therefore new line can be enter typing a sentence in a new line or using \n'''
print("""•\tPassengerId: Id of every passenger.\n•\tSurvived: This feature have value 0 and 1. 0 for not survived and 1 for survived.
•\tPclass: There are 3 classes: Class 1, Class 2 and Class 3.\n•\tName: Name of passenger.\n•\tSex: Gender of passenger.
•\tAge: Age of passenger.\n•\tSibSp: Indication that passenger have siblings and spouse.\n•\tParch: Whether a passenger is alone or have family.
•\tTicket: Ticket number of passenger.\n•\tFare: Indicating the fare.\n•\tCabin: The cabin of passenger.\n•\tEmbarked: The embarked category""")

	Data description
*****************************************
•	PassengerId: Id of every passenger.
•	Survived: This feature have value 0 and 1. 0 for not survived and 1 for survived.
•	Pclass: There are 3 classes: Class 1, Class 2 and Class 3.
•	Name: Name of passenger.
•	Sex: Gender of passenger.
•	Age: Age of passenger.
•	SibSp: Indication that passenger have siblings and spouse.
•	Parch: Whether a passenger is alone or have family.
•	Ticket: Ticket number of passenger.
•	Fare: Indicating the fare.
•	Cabin: The cabin of passenger.
•	Embarked: The embarked category


## Transforming the column values
### Transforming Survived and Pclass columns from integer to String
Values are transformed form numerical values to categotical values as it is easier to understand the data in the exploratary analysis.
Also, after the transformation, graph lables become more discriptive and easier to understand.

### Renaming Embarked column values to their proper Embarcation port
This will make data more understandable in data exploration.


In [5]:
# Survived column values: maped 1 -> Survived and 0 -> Deceased
train_df.loc[train_df['Survived'] == 0, 'Survived'] = 'Deceased' 
train_df.loc[train_df['Survived'] == 1, 'Survived'] = 'Survived' 
train_df['Survived'].head()

0    Deceased
1    Survived
2    Survived
3    Survived
4    Deceased
Name: Survived, dtype: object

In [6]:
# Pclass column values: maped 1 -> First Class, 2 -> Second Class and 3 -> Third Class
train_df.loc[train_df['Pclass'] == 1, 'Pclass'] = 'First Class' 
train_df.loc[train_df['Pclass'] == 2, 'Pclass'] = 'Second Class' 
train_df.loc[train_df['Pclass'] == 3, 'Pclass'] = 'Third Class' 
train_df['Pclass'].head()

0    Third Class
1    First Class
2    Third Class
3    First Class
4    Third Class
Name: Pclass, dtype: object

In [7]:
# Embarked column values: maped C -> Cherbourg, Q -> Queenstown and S -> Southampton
train_df.loc[train_df['Embarked'] == 'C', 'Embarked'] = 'Cherbourg' 
train_df.loc[train_df['Embarked'] == 'Q', 'Embarked'] = 'Queenstown' 
train_df.loc[train_df['Embarked'] == 'S', 'Embarked'] = 'Southampton'
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,Deceased,Third Class,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,Southampton
1,2,Survived,First Class,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,Cherbourg
2,3,Survived,Third Class,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,Southampton
3,4,Survived,First Class,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,Southampton
4,5,Deceased,Third Class,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,Southampton


In [8]:
#The average age of passengers? 
train_df["Age"].mean()

29.69911764705882

In [9]:
#TODO%%%%%%%%%%%% WE NEED CHECH THIS @Bassey
# Median Age and Ticket fare price of Pax
train_df[["Age", "Fare"]].median()
train_df.describe(include="all")

#instead of using the predifined "describe function" we can combined aggregating statistics 
train_df.agg( 
    { 
        "Age": ['min', 'max', 'mean', 'median', 'skew'], 
        "Fare": ['min', 'max', 'mean', 'median', 'skew'], 
    } )

,Age,Fare
min,0.420000,0.000000
max,80.000000,512.329200
mean,29.699118,32.204208
median,28.000000,14.454200
skew,0.389108,4.787317


In [10]:
#counting by group
train_df.groupby("Pclass")["Pclass"].count()

Pclass
First Class     216
Second Class    184
Third Class     491
Name: Pclass, dtype: int64

In [11]:
train_df.isnull()  # is any missing values in dataframe
train_df.isnull().any()  # is any missing values across columns
train_df.isnull().sum().sum()   # count of missing values of the entire dataframe
count_NAN = len(train_df) - train_df.count() # count of missing values across columns
count_NAN

for i in range(len(train_df.index)):  # count of missing values across rows
    print("NAN in row ", i , " : ", train_df.iloc[i].isnull().sum())


NAN in row  0  :  1
NAN in row  1  :  0
NAN in row  2  :  1
NAN in row  3  :  0
NAN in row  4  :  1
NAN in row  5  :  2
NAN in row  6  :  0
NAN in row  7  :  1
NAN in row  8  :  1
NAN in row  9  :  1
NAN in row  10  :  0
NAN in row  11  :  0
NAN in row  12  :  1
NAN in row  13  :  1
NAN in row  14  :  1
NAN in row  15  :  1
NAN in row  16  :  1
NAN in row  17  :  2
NAN in row  18  :  1
NAN in row  19  :  2
NAN in row  20  :  1
NAN in row  21  :  0
NAN in row  22  :  1
NAN in row  23  :  0
NAN in row  24  :  1
NAN in row  25  :  1
NAN in row  26  :  2
NAN in row  27  :  0
NAN in row  28  :  2
NAN in row  29  :  2
NAN in row  30  :  1
NAN in row  31  :  1
NAN in row  32  :  2
NAN in row  33  :  1
NAN in row  34  :  1
NAN in row  35  :  1
NAN in row  36  :  2
NAN in row  37  :  1
NAN in row  38  :  1
NAN in row  39  :  1
NAN in row  40  :  1
NAN in row  41  :  1
NAN in row  42  :  2
NAN in row  43  :  1
NAN in row  44  :  1
NAN in row  45  :  2
NAN in row  46  :  2
NAN in row  47  :  2
NA

In [12]:
#counting of missing values of a particular column
train_df.Age.isnull().sum()

177

In [13]:
# count of missing values of column by group
'''count of missing values of column by group
 Because we dont havve missing values in sex all the missing values is coming from Age'''
train_df.groupby(['Sex'])['Survived'].apply(lambda x: x.isnull().sum())

Sex
female    0
male      0
Name: Survived, dtype: int64

In [14]:
# Replacing the na in age column with mean
train_df["Age"] = train_df["Age"].fillna(round(train_df["Age"].mean()))
train_df["Age"].head()

#checking if there are any remaining missing value
train_df.Age.isnull().sum()

0

In [15]:
# Percentage of rows missing in each column

for column in train_df.columns:
    percentage = train_df[column].isnull().mean()
    print(f'{column}: {round(percentage*100,4)}%')

PassengerId: 0.0%
Survived: 0.0%
Pclass: 0.0%
Name: 0.0%
Sex: 0.0%
Age: 0.0%
SibSp: 0.0%
Parch: 0.0%
Ticket: 0.0%
Fare: 0.0%
Cabin: 77.1044%
Embarked: 0.2245%
